In [1]:
import numpy as np
print('numpy version:', np.__version__)


numpy version: 1.23.5


Another Example of Value Iteration (Software Implementation)  

Consider the grid with reward:  
<img src="images/unit5_value_update_algo.png" />    
However, consider the following change to the transition probabilities: At any given grid location the agent can choose to either stay at the location or move to an adjacent grid location. If the agent chooses to stay at the location, such an action is successful with probability 1/2 and
- if the agent is at the leftmost or rightmost grid location it ends up at its neighboring grid location with probability 1/2,
- if the agent is at any of the inner grid locations it has a probability 1/4 each of ending up at either of the neighboring locations.
  
If the agent chooses to move (either left or right) at any of the inner grid locations, such an action is successful with probability 1/3 and with probability 2/3 it fails to move, and
- if the agent chooses to move left at the leftmost grid location, then the action ends up exactly the same as choosing to stay, i.e., staying at the leftmost grid location with probability 1/2, and ends up at its neighboring grid location with probability 1/2,
- if the agent chooses to move right at the rightmost grid location, then the action ends up exactly the same as choosing to stay, i.e., staying at the rightmost grid location with probability 1/2, and ends up at its neighboring grid location with probability 1/2.
  
[!!!] Note in this setting, we assume that the game does not halt after reaching the rightmost cell.
  
Let $\gamma = 0.5$.  
  
Run the value iteration algorithm for 100 iterations. Use any computational software of your choice.

In [23]:
S = ['s1', 's2', 's3', 's4', 's5']
A = ['stay', 'left', 'right']
T = np.array(
    [
        [[.5, .5, 0., 0., 0.], # a = 'stay', rows = s, columns = s'
         [.25, .5, .25, 0., 0.],
         [0., .25, .5, .25, 0.],
         [0., 0., .25, .5, .25],
         [0., 0., 0., .5, .5]],
        [[.5, .5, 0., 0., 0.], # a = 'left', rows = s, columns = s'
         [1/3, 2/3, 0., 0., 0.],
         [0., 1/3, 2/3, 0., 0.],
         [0., 0., 1/3, 2/3, 0.],
         [0., 0., 0., 1/3, 2/3]],
        [[2/3, 1/3, 0., 0., 0.], # a = 'right', rows = s, columns = s'
         [0., 2/3, 1/3, 0., 0.],
         [0., 0., 2/3, 1/3, 0.],
         [0., 0., 0., 2/3, 1/3],
         [0., 0., 0., .5, .5]]
    ]
)
R = np.array([0., 0., 0., 0., 1.])
V = np.zeros((5,))
gamma = 0.5

In [24]:
# !!! R relates to current_state while V relates to future_state
reward = R.reshape(-1, 1) + gamma * V.reshape(1, -1)
reward

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.]])

In [25]:
T * reward

array([[[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.5       , 0.5       ]],

       [[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.33333333, 0.66666667]],

       [[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.     

In [26]:
(T * reward).sum(axis=2)

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [27]:
(T * reward).sum(axis=2).max(axis=0)

array([0., 0., 0., 0., 1.])

In [28]:
def update_v(V, T, R):
    """
    Returns updated value array (V)

    """
    reward = R.reshape(-1, 1) + gamma * V.reshape(1, -1) # computing reward part (R(s) + gamma * V_k(s')) !!! it's IMPORTANT that s' !!!
    reward_lh = T * reward # computing reward weighted by likelihood (probabilities) for each current_state, future_state and action => shape (3, 5, 5)
    Q = reward_lh.sum(axis=2) # summing up across last dimension (future_states) => shape (3, 5) actions x current_states
    V_new = Q.max(axis=0) # determining best variant for each state across possible actions => shape (5,)
    return V_new

In [29]:
V

array([0., 0., 0., 0., 0.])

In [30]:
V_upd = update_v(V=V, T=T, R=R)
V_upd

array([0., 0., 0., 0., 1.])

In [31]:
V_upd = update_v(V=V_upd, T=T, R=R)
V_upd

array([0.        , 0.        , 0.        , 0.16666667, 1.33333333])

It seems working. Let's use loop to end up with solution

In [32]:
for _ in range(100):
    V = update_v(V=V, T=T, R=R)
V

array([0.00833333, 0.025     , 0.1       , 0.4       , 1.6       ])

In [33]:
# iterate 100 cycles more to compare results
for _ in range(100):
    V = update_v(V=V, T=T, R=R)
V

array([0.00833333, 0.025     , 0.1       , 0.4       , 1.6       ])

In [34]:
V = np.zeros((5,))
for _ in range(10):
    V = update_v(V=V, T=T, R=R)
V

array([0.00761459, 0.02415681, 0.09904883, 0.39902345, 1.59902343])

In [21]:
import numpy as np
state = [0, 1, 2, 3, 4]
action = [0, 1, 2] #, representing moving left, staying, moving right respectively 
#transition probability
T = np.array(
    [
        [[1/2,1/2,0,0,0],
         [1/2,1/2,0,0,0],
         [2/3,1/3,0,0,0]],
        [[1/3,2/3,0,0,0],
         [1/4,1/2,1/4,0,0],
         [0,2/3,1/3,0,0]],
        [[0,1/3,2/3,0,0],
         [0,1/4,1/2,1/4,0],
         [0,0,2/3,1/3,0]],
        [[0,0,1/3,2/3,0],
         [0,0,1/4,1/2,1/4],
         [0,0,0,2/3,1/3]],
        [[0,0,0,1/3,2/3],
         [0,0,0,1/2,1/2],
         [0,0,0,1/2,1/2]]
    ])
num_state = 5
num_action = 3
r = 1/2
# initialization
V = np.zeros(5)
# reward
R = np.zeros(5)
R[4] = 1
num_iter = 10
for i in range(num_iter):
    Q = [[sum([T[s][a][t] * (R[s] + r * V[t]) for t in range(num_state)]) for a in range(num_action)] for s in range(num_state)]
    V = np.max(Q, axis=1)
print(V)

[0.00761459 0.02415681 0.09904883 0.39902345 1.59902343]
